In [1]:
from tensorflow import keras

2023-11-10 15:01:26.069721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 15:01:26.069759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 15:01:26.070520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 15:01:26.171976: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 15:01:27.557560: W tensorflow/compiler/

In [6]:
model = keras.Sequential()
model.add(keras.layers.Dense(64, activation=keras.activations.relu))
model.add(keras.layers.Dense(10, activation=keras.activations.softmax))

In [7]:
model.build(input_shape=(None, 3))
model.weights

[<tf.Variable 'dense_1/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[ 0.14846009, -0.07598916,  0.08779973, -0.15998262, -0.05323645,
         -0.13413858,  0.16024682, -0.13156958,  0.09894413, -0.22456032,
         -0.27154532,  0.19918755, -0.0976284 ,  0.19865492,  0.2518701 ,
          0.05468294,  0.25761276, -0.06505729, -0.17490123, -0.15660839,
         -0.07110527, -0.2596689 , -0.21981822,  0.02241907,  0.2386592 ,
          0.21783304,  0.29615366, -0.2764152 ,  0.09915018, -0.29382578,
          0.2336387 , -0.28709793,  0.19570836, -0.06828976,  0.14711896,
         -0.1233906 ,  0.10971692,  0.06639108, -0.18250023,  0.1990943 ,
         -0.15570806, -0.024295  , -0.19035316, -0.08308658, -0.13963012,
          0.25900728, -0.18779606, -0.149806  , -0.19189006, -0.28744045,
         -0.0807441 ,  0.09780428, -0.18353292,  0.25570697,  0.24306464,
         -0.2504045 ,  0.09138265, -0.21410328,  0.17069417,  0.13438067,
         -0.09679756, -0.08473684, -0.18210

In [9]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                256       
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Declaring names for the model and its layers

In [14]:
model = keras.Sequential(name="the_example_model")
model.add(keras.layers.Dense(64, activation=keras.activations.relu, name="the_first_layer"))
model.add(keras.layers.Dense(10, activation=keras.activations.softmax, name="the_second_layer"))

In [15]:
model.build(input_shape=(None, 3))

In [16]:
model.summary()

Model: "the_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_first_layer (Dense)     (None, 64)                256       
                                                                 
 the_second_layer (Dense)    (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Declaring the shape of the inputs of the model in advance

In [19]:
model = keras.Sequential()
model.add(keras.Input(shape=(3, ))) # the shape argument here represents the shape of each sample, not one batch
model.add(keras.layers.Dense(64, keras.activations.relu))

In [20]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
Total params: 256 (1.00 KB)
Trainable params: 256 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.add(keras.layers.Dense(10, activation=keras.activations.softmax))

In [23]:
model.summary() # Checking properties of the model without using the buold function

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 7.2.2 The Functional API

In [25]:
inputs = keras.Input(shape=(3, ), name="my_input")
features = keras.layers.Dense(64, activation=keras.activations.relu)(inputs)
outputs = keras.layers.Dense(10, activation=keras.activations.relu)(features)
model = keras.Model(inputs=inputs, outputs=outputs)

#### Multi-input, multi-output models

In [26]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

title = keras.Input(shape=(vocabulary_size, ), name="title")
text_body = keras.Input(shape=(vocabulary_size, ), name="text_body")
tags = keras.Input(shape=(num_tags, ), name="tags")

features = keras.layers.Concatenate()([title, text_body, tags])
features = keras.layers.Dense(64, activation=keras.activations.relu)(features)

priority = keras.layers.Dense(1, activation=keras.activations.sigmoid, name="priority")(features)
department = keras.layers.Dense(num_departments, activation=keras.activations.softmax, name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

#### Training a multi-input, multi-output model

In [28]:
import numpy as np

num_samples = 1280

# Dummy input data
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

# Dummy target data
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [48]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss={"priority": keras.losses.mean_squared_error, "department": keras.losses.categorical_crossentropy},
    metrics={"priority": [keras.metrics.mean_absolute_error], "department": [keras.metrics.Accuracy()]}
)

##### Writing the inputs, outputs etc. as a dictionary instead of a list gives flexibility to assign names for each data

In [49]:
model.fit(
    {"title" : title_data, "text_body" : text_body_data, "tags" : tags_data},
    {"priority" : priority_data, "department" : department_data},
    epochs=1
)

40/40 [==============================] - 1s 21ms/step - loss: 65.8547 - priority_loss: 0.3356 - department_loss: 65.5191 - priority_mean_absolute_error: 0.5040 - department_accuracy: 0.0365


In [50]:
model.evaluate(
    {"title" : title_data, "text_body" : text_body_data, "tags" : tags_data},
    {"priority" : priority_data, "department" : department_data}
)

40/40 [==============================] - 0s 6ms/step - loss: 65.0220 - priority_loss: 0.3356 - department_loss: 64.6864 - priority_mean_absolute_error: 0.5040 - department_accuracy: 0.0000e+00


[65.02201843261719,
 0.3356093764305115,
 64.6864013671875,
 0.5040120482444763,
 0.0]